In [14]:
# psycopg
import psycopg2
import pandas as pd
import numpy as np
import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [15]:
# csv file
csvfile1 = "listings.csv"
csvfile2 = "crime_2019.csv"
csvfile3 = "reviews.csv"

In [16]:
# read csv file into pandas dataframe
listings_df = pd.read_csv(csvfile1)
crime_df = pd.read_csv(csvfile2)
reviews_df = pd.read_csv(csvfile3)

In [17]:
# Create an engine to the postgres database

engine = create_engine("sqlite:///airbnb_crime_dataset.sqlite")

In [18]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [19]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options

listings_data = listings_df.to_dict(orient='records')
crime_data = crime_df.to_dict(orient='records')
reviews_data = reviews_df.to_dict(orient='records')


listings_data[1]

{'id': 360,
 'host_name': 'Jennifer & Giovanni',
 'host_response_time': 'within an hour',
 'host_is_superhost': 't',
 'neighbourhood': 'Highland',
 'neighbourhood_cleansed': 'Highland',
 'zipcode': 80211.0,
 'latitude': 39.76703,
 'longitude': -105.00256,
 'property_type': 'Guesthouse',
 'room_type': 'Entire home/apt',
 'accommodates': 5,
 'bathrooms': 1.0,
 'bedrooms': 2.0,
 'beds': 2.0,
 'square_feet': nan,
 'price': '$140.00 ',
 'number_of_reviews': 48,
 'last_review': '3/29/19',
 'review_scores_rating': 100.0}

In [20]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [21]:
# Use `declarative_base` from SQLAlchemy to model the listing table as an ORM class
Base = declarative_base()

class Listings(Base):
    __tablename__ = 'listings'

    id = Column(Integer, primary_key=True)
    host_name = Column(String(255))
    host_response_time = Column(String(255))
    host_is_superhost = Column(String(255))
    neighbourhood = Column(String(255))
    neighbourhood_cleansed = Column(String(255))
    zipcode = Column(Float)
    latitude = Column(Float)
    longitude = Column(Float)
    property_type = Column(String(255))
    room_type = Column(String(255))
    accommodates = Column(Integer)
    bathrooms = Column(Float)
    bedrooms = Column(Float)
    beds = Column(Float)
    square_feet = Column(String(255))
    price = Column(String(255))
    number_of_reviews = Column(Integer)
    last_review = Column(String(255))
    review_scores_rating = Column(Float)
    
    
#     def __repr__(self):
#         return f"id={self.id}, name={self.name}"

In [28]:
# Use `declarative_base` from SQLAlchemy to model the listing table as an ORM class
Base = declarative_base()

class Crime(Base):
    __tablename__ = 'crime'

    id = Column(Integer, primary_key=True)
    OFFENSE_CATEGORY_ID = Column(String(255))
    GEO_LON = Column(Float)
    GEO_LAT = Column(Float)
    NEIGHBORHOOD_ID = Column(String(255))
    
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [38]:
Base = declarative_base()

class Reviews(Base):
    __tablename__ = 'reviews'

    id = Column(Integer, primary_key=True)
    listing_id = Column(Integer)
    date = Column(String(255))
    comments = Column(String(1024))
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [39]:
# Use `create_all` to create the demographics table in the database
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [40]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [24]:
listings_table = sqlalchemy.Table('listings', metadata, autoload=True)

In [32]:
crime_table = sqlalchemy.Table('crime', metadata, autoload=True)

In [41]:
reviews_table = sqlalchemy.Table('reviews', metadata, autoload=True)

In [25]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
### BEGIN SOLUTION
conn.execute(listings_table.delete())
### END SOLUTION

In [33]:
conn.execute(crime_table.delete())

In [42]:
conn.execute(reviews_table.delete())

In [26]:
# Use `table.insert()` to insert the data into the table
### BEGIN SOLUTION
conn.execute(listings_table.insert(), listings_data)
### END SOLUTION

In [34]:
conn.execute(crime_table.insert(), crime_data)

In [43]:
conn.execute(reviews_table.insert(), reviews_data)

In [45]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from listings limit 5").fetchall()

[(177, 'Joe', 'within an hour', 't', 'Virginia Village', 'Virginia Village', 80246.0, 39.69585, -104.92582, 'Guesthouse', 'Entire home/apt', 2, 1.0, 1.0, 1.0, None, '$56.00 ', 39, '3/24/19', 94.0),
 (360, 'Jennifer & Giovanni', 'within an hour', 't', 'Highland', 'Highland', 80211.0, 39.76703, -105.00256, 'Guesthouse', 'Entire home/apt', 5, 1.0, 2.0, 2.0, None, '$140.00 ', 48, '3/29/19', 100.0),
 (590, 'Jill', 'within an hour', 't', 'North Park Hill', 'North Park Hill', 80207.0, 39.753609999999995, -104.91237, 'House', 'Private room', 3, 1.0, 1.0, 1.0, None, '$61.00 ', 565, '3/27/19', 97.0),
 (592, 'Jill', 'within an hour', 't', 'North Park Hill', 'North Park Hill', 80207.0, 39.75438, -104.91156000000001, 'House', 'Private room', 2, 1.0, 1.0, 1.0, None, '$42.00 ', 147, '3/22/19', 97.0),
 (686, 'Alexandra', 'within an hour', 'f', 'North Capitol Hill', 'North Capitol Hill', 80203.0, 39.74503, -104.97756000000001, 'House', 'Private room', 2, 2.0, 1.0, 1.0, None, '$70.00 ', 236, '4/16/19', 

In [46]:
conn.execute("select * from crime limit 5").fetchall()

[(1, 'robbery', -104.8887503, 39.7398543, 'east-colfax'),
 (2, 'auto-theft', -105.03111899999999, 39.76453970000001, 'west-highland'),
 (3, 'larceny', -104.9590553, 39.7403303, 'city-park'),
 (4, 'public-disorder', -104.825985, 39.797799100000006, 'montbello'),
 (5, 'drug-alcohol', -105.0153483, 39.730808, 'lincoln-park')]

In [47]:

conn.execute("select * from reviews limit 5").fetchall()

[(1, 177, '9/4/16', 'Great place to stay! Definitely a tiny home, but still cozy and comfortable. Pretty fun to try it out for a couple nights for the two of us. Joe was helpful and friendly. Destinations were always less than a 20min drive and lots of parks nearby! Super fun stay :)'),
 (2, 177, '9/13/16', "Joe was very friendly and gave us all the information we needed. The house was exactly as described on the site. It was fun and thought provoking experiencing a 100% Eco-friendly space. We plan on staying here again next time we're in Denver!"),
 (3, 177, '9/14/16', 'Amazing property and Joe was a great host! Even offered to walk my dog while we were out! Will definitely be back'),
 (4, 177, '9/19/16', 'Joe was a great host, the description of this tiny house was spot on.  It was definitely a small space but well laid out and very convenient.  I would recommend a stay there.'),
 (5, 177, '9/22/16', 'He welcomes us when we first arrived and gave great recommendations for dinner/brea